In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
import re
data = pd.merge(movies,ratings)
data

,movie_id,title,genre,user_id,count,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474
...,...,...,...,...,...,...
836473,3952,"Contender, The (2000)",Drama|Thriller,5682,3,1029457829
836474,3952,"Contender, The (2000)",Drama|Thriller,5812,4,992072099
836475,3952,"Contender, The (2000)",Drama|Thriller,5831,3,986223125
836476,3952,"Contender, The (2000)",Drama|Thriller,5837,4,1011902656


분석
1. ratings에 있는 유니크한 영화 개수
2. rating에 있는 유니크한 사용자 수
3. 가장 인기있는 영화 30개

1. ratings에 있는 유니크한 영화 개수

In [7]:
ratings['movie_id'].nunique()

3628

2. rating에 있는 유니크한 사용자 수

In [8]:
ratings['user_id'].nunique()

6039

3. 가장 인기있는 영화 30개

In [9]:
popular = data.groupby('title')['user_id'].count()
popular.sort_values(ascending=False).head(40)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

선호영화5개 추가

In [10]:
my_favorite = ['Fargo (1996)', 'Blade Runner (1982)', 'Aliens (1986) ', 'Terminator, The (1984)', 'Matrix, The (1999)']
my_list = pd.DataFrame({'user_id': ['teaho']*5, 'title': my_favorite, 'count':[5]*5})

my_list

,user_id,title,count
0,teaho,Fargo (1996),5
1,teaho,Blade Runner (1982),5
2,teaho,Aliens (1986),5
3,teaho,"Terminator, The (1984)",5
4,teaho,"Matrix, The (1999)",5


In [11]:
#data = pd.concat([mratings,my_list], ignore_index=True)

if not data.isin({'user_id':['teaho']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_list)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,movie_id,title,genre,user_id,count,timestamp
836473,3952.0,"Contender, The (2000)",Drama|Thriller,5682,3,1.029458e+09
836474,3952.0,"Contender, The (2000)",Drama|Thriller,5812,4,9.920721e+08
836475,3952.0,"Contender, The (2000)",Drama|Thriller,5831,3,9.862231e+08
836476,3952.0,"Contender, The (2000)",Drama|Thriller,5837,4,1.011903e+09
836477,3952.0,"Contender, The (2000)",Drama|Thriller,5998,4,1.001781e+09
0,NaN,Fargo (1996),NaN,teaho,5,NaN
1,NaN,Blade Runner (1982),NaN,teaho,5,NaN
2,NaN,Aliens (1986),NaN,teaho,5,NaN
3,NaN,"Terminator, The (1984)",NaN,teaho,5,NaN
4,NaN,"Matrix, The (1999)",NaN,teaho,5,NaN


In [12]:
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [13]:
print(user_to_idx['teaho'])

6039


In [14]:
print(title_to_idx['Aliens (1986)'])

1084


In [15]:
print(len(data.user_id))

836483


In [16]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

user_id column indexing OK!!
title column indexing OK!!


CSR matrix

In [17]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data['count'], (data.user_id, data.title)), shape=(num_user, num_title))
csr_data

<6040x3629 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

als_model=AlternatingLeastSquares 모델구성, 훈련

In [18]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [19]:
teaho, aliens = user_to_idx['teaho'], title_to_idx['Aliens (1986)']
teaho_vector, aliens_vector = als_model.user_factors[teaho], als_model.item_factors[aliens]

print('슝=3')

슝=3


In [20]:
teaho_vector

array([-6.32962048e-01,  5.44142611e-02, -6.92825168e-02, -5.88229060e-01,
        8.08740139e-01,  1.12891272e-01, -3.98223512e-02,  1.14055514e-01,
        1.25753498e+00, -1.70315489e-01, -1.64924026e-01,  5.78678489e-01,
        3.99129651e-02, -9.81498778e-01, -2.96569288e-01,  1.13870609e+00,
       -3.92320484e-01,  1.11805826e-01, -6.20780289e-01, -1.84443399e-01,
       -4.59895670e-01, -3.08642685e-01, -7.41392791e-01,  3.77304673e-01,
       -3.43237519e-01,  2.26581544e-01,  1.81903362e-01,  1.14974633e-01,
       -6.17192745e-01,  3.04404050e-01, -6.94962859e-01, -4.93704021e-01,
       -4.00430948e-01,  3.62439118e-02,  2.86864698e-01,  9.00524914e-01,
        2.08348528e-01, -8.27113818e-03, -5.83767593e-01,  5.34408867e-01,
       -5.08309662e-01, -3.16696912e-01, -1.41929314e-01, -7.75085211e-01,
       -6.71468973e-01, -8.46500918e-02,  3.46489847e-02, -2.75933444e-01,
       -5.12993455e-01, -1.28163680e-01,  6.64613903e-01,  2.78801862e-02,
        2.80378317e-03, -

In [21]:
aliens_vector

array([-8.7120803e-03, -1.3919465e-02, -9.9703018e-03,  5.5399621e-03,
        9.7521897e-03,  6.5148796e-04,  2.2213154e-02, -1.1974742e-02,
        2.2704085e-02, -3.8236710e-03,  6.8356427e-03,  4.3832749e-02,
        8.4907534e-03,  3.9268387e-03, -1.8541943e-02,  4.5455374e-02,
       -7.9215867e-03,  2.8638598e-02, -1.7714893e-02,  1.2109762e-02,
        2.1602813e-04,  6.7283452e-04, -2.2125166e-02,  1.4883179e-02,
        2.3560507e-02,  2.0799704e-03,  1.8053267e-02, -7.0828022e-03,
        7.0760916e-06,  1.6603759e-02, -2.6756080e-02, -1.4105089e-02,
        7.0968838e-03, -2.3350643e-02,  2.8288146e-02,  2.5281061e-03,
        1.7096560e-02,  1.1923815e-02, -2.8003769e-02,  1.3983742e-04,
       -4.1504768e-03,  1.3758725e-03,  2.0060672e-03, -1.1482124e-02,
       -8.8567035e-03, -1.3574733e-03,  2.2125650e-02,  1.5176371e-03,
        2.0175949e-02,  1.9665889e-02,  8.2077477e-03,  9.3755554e-03,
        7.0958771e-03, -1.2092919e-03,  2.9846242e-02,  3.3326011e-02,
      

In [22]:
np.dot(teaho_vector, aliens_vector)

0.4481493

In [23]:
contender = title_to_idx['Contender, The (2000)']
contender_vector = als_model.item_factors[contender]
np.dot(teaho_vector, contender_vector)

-0.019448828

비슷한 영화추천

In [24]:
favorite_movie = 'Toy Story (1995)'
movie_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(0, 0.9999999),
 (2845, 0.80237955),
 (33, 0.60631084),
 (568, 0.59621763),
 (2114, 0.57047254),
 (1147, 0.555772),
 (2081, 0.48329422),
 (350, 0.4821442),
 (575, 0.43619478),
 (1705, 0.42823383),
 (2155, 0.4006691),
 (1409, 0.3772594),
 (342, 0.37385327),
 (2434, 0.37350702),
 (304, 0.36600363)]

In [25]:
#movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 'Babe (1995)',
 'Aladdin (1992)',
 "Bug's Life, A (1998)",
 'Groundhog Day (1993)',
 'Pleasantville (1998)',
 'Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 "There's Something About Mary (1998)",
 'Shakespeare in Love (1998)',
 'Hercules (1997)',
 'Forrest Gump (1994)',
 'Tarzan (1999)',
 'Santa Clause, The (1994)']

In [26]:
title_to_idx

{'Toy Story (1995)': 0,
 'Jumanji (1995)': 1,
 'Grumpier Old Men (1995)': 2,
 'Waiting to Exhale (1995)': 3,
 'Father of the Bride Part II (1995)': 4,
 'Heat (1995)': 5,
 'Sabrina (1995)': 6,
 'Tom and Huck (1995)': 7,
 'Sudden Death (1995)': 8,
 'GoldenEye (1995)': 9,
 'American President, The (1995)': 10,
 'Dracula: Dead and Loving It (1995)': 11,
 'Balto (1995)': 12,
 'Nixon (1995)': 13,
 'Cutthroat Island (1995)': 14,
 'Casino (1995)': 15,
 'Sense and Sensibility (1995)': 16,
 'Four Rooms (1995)': 17,
 'Ace Ventura: When Nature Calls (1995)': 18,
 'Money Train (1995)': 19,
 'Get Shorty (1995)': 20,
 'Copycat (1995)': 21,
 'Assassins (1995)': 22,
 'Powder (1995)': 23,
 'Leaving Las Vegas (1995)': 24,
 'Othello (1995)': 25,
 'Now and Then (1995)': 26,
 'Persuasion (1995)': 27,
 'City of Lost Children, The (1995)': 28,
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)': 29,
 'Dangerous Minds (1995)': 30,
 'Twelve Monkeys (1995)': 31,
 'Wings of Courage (1995)': 32,
 'Babe (1995)'

In [27]:
def get_similar_movie(movie_name: str):
    movie_id = title_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [28]:
get_similar_movie('Aliens (1986)')

['Aliens (1986)',
 'Alien (1979)',
 'Terminator, The (1984)',
 'Predator (1987)',
 'Blade Runner (1982)',
 'Abyss, The (1989)',
 'Die Hard (1988)',
 'Mad Max 2 (a.k.a. The Road Warrior) (1981)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Full Metal Jacket (1987)']

좋아할만한 영화 추천

In [29]:
user = user_to_idx['teaho']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(569, 0.5318749),
 (573, 0.45203942),
 (1084, 0.4481493),
 (1098, 0.43372512),
 (1455, 0.41372156),
 (49, 0.37121654),
 (2657, 0.31687427),
 (305, 0.31535545),
 (283, 0.30630517),
 (1080, 0.2803231),
 (249, 0.27656126),
 (31, 0.2649575),
 (949, 0.25907964),
 (843, 0.2578874),
 (3224, 0.25057024),
 (1083, 0.24275604),
 (1090, 0.23802349),
 (1097, 0.23127592),
 (3387, 0.23081672),
 (995, 0.21869585)]

In [30]:
[idx_to_title[i[0]] for i in movie_recommended]

['Terminator 2: Judgment Day (1991)',
 'Silence of the Lambs, The (1991)',
 'Aliens (1986)',
 'Alien (1979)',
 'L.A. Confidential (1997)',
 'Usual Suspects, The (1995)',
 'Total Recall (1990)',
 'Shawshank Redemption, The (1994)',
 'Pulp Fiction (1994)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Twelve Monkeys (1995)',
 'Die Hard (1988)',
 '2001: A Space Odyssey (1968)',
 'Predator (1987)',
 'Brazil (1985)',
 'Clockwork Orange, A (1971)',
 'GoodFellas (1990)',
 'Mad Max 2 (a.k.a. The Road Warrior) (1981)',
 'Reservoir Dogs (1992)']

In [31]:
die = title_to_idx['Die Hard (1988)']
explain = als_model.explain(user, csr_data, itemid=die)

In [32]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Terminator, The (1984)', 0.2136855813729771),
 ('Matrix, The (1999)', 0.04642031196704568),
 ('Blade Runner (1982)', 0.012853931084282197),
 ('Aliens (1986) ', 0.0007652586967953705),
 ('Fargo (1996)', -0.017246780757759535)]